In [1]:
import numpy as np

%pwd

'/data/renqihan/interaction_code_base/src'

In [2]:
import os
os.system("curl https://www.google.com")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0

<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="zh-HK"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="CUnQUo7HLlgrssm22CUwlA">(function(){var _g={kEI:'rVegZ735Oemivr0P98eD8A8',kEXPI:'0,202746,606135,2891417,1086,538661,2872,2891,43028,30022,16105,344796,212051,26408,20674,30911,15671,5225084,766,8834933,23934161,4043709,25228681,117307,5797,15164,8181,49430,21666,6758,23878,9139,4600,328,6225,1117,26873,5633,687,29855,1346,13703,15634,12129,2392,3290,3491,33,4678,4363,17667,10672,14559,11267,3853,41,2956,2651,7556,476,1,1720,5021,4105,307,4148,8994,5853,839,967,4946,621,1525,761,3856,5774,4310,2326,1310,738,4721,78,1545,1623,143,1207,9610,820,101,3798,1633,1,237,5446,1832,621,1904,131,233,1864,3,3,698,460,41,2,416,1845,91,595,35,2094,172,6,755,393,714,1768,1275,55,560,464,1,215,612,1,81,140,1,37,667,1895

100 19779    0 19779    0     0   6097      0 --:--:--  0:00:03 --:--:--  6097


0

In [3]:
from utils.global_const import *

# Set the cache directory for huggingface hub. [Important!] It should be before the import of transformers
os.environ["HF_HOME"] = HF_HOME

from interaction.player import get_player_ids_from_word
from utils import get_dataset_nlp

from transformers import AutoTokenizer
from utils import load_json, save_json

# dataset_name = "custom-squad-v1-20250126-train"
# dataset_name = "custom-squad-v1-20250126-val"

# dataset_name = "custom-squad-v2-20250202-train"
dataset_name = "custom-squad-v2-20250202-val"

/home/renqihan/.conda/envs/sae/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Get player ids for **Pythia** model family.

Use the pre-selected player words from `preprocess_squad.ipynb`.

In [7]:
tokenizer_pythia = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m", add_prefix_space=True)
# todo: remember to add add_prefix_space=True!!

In [8]:
dataset_pythia = get_dataset_nlp(dataset_name=dataset_name,
                                 mode="eval",
                                 tokenizer=tokenizer_pythia,
                                 batch_size=1,
                                 data_path=f"../datasets/{dataset_name}")
dataset_pythia.load_data()
sentences = np.array(dataset_pythia.dataset_raw["text"])
save_dir_pythia = f"../players/{dataset_name}/players-pythia"

print(len(sentences))

Map: 100%|██████████| 1434/1434 [00:01<00:00, 1038.66 examples/s]

1434


In [9]:
player_words_all_with_idx = load_json(os.path.join(save_dir_pythia, 'player_words.json'))
player_words_all = list(player_words_all_with_idx.values())
sample_indices = [int(s) for s in player_words_all_with_idx.keys()]

assert len(player_words_all) == len(sample_indices) == len(sentences)

player_ids_all = get_player_ids_from_word(tokenizer=tokenizer_pythia,
                                            sentences=sentences[sample_indices],
                                            player_words_all=player_words_all,
                                            save_dir=None)
assert len(player_ids_all) == len(sample_indices)

player_ids_all_with_idx = {f"{idx}": player_ids for idx, player_ids in zip(sample_indices, player_ids_all)}
save_json(player_ids_all_with_idx, save_dir_pythia, 'player_ids_from_word.json')

Get player ids for **Qwen2.5** model family.

Use the pre-selected player words from `preprocess_squad.ipynb`.

In [11]:
from models.nlp import Qwen2TokenizerModified

tokenizer_qwen = Qwen2TokenizerModified.from_pretrained("Qwen/Qwen2.5-1.5B", add_prefix_space=True)
# todo: remember to add add_prefix_space=True!!

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Qwen2Tokenizer'. 
The class this function is called from is 'Qwen2TokenizerModified'.


In [12]:
print(tokenizer_qwen.pad_token)
print(tokenizer_qwen.pad_token_id)
print(tokenizer_qwen.decode(tokenizer_qwen.pad_token_id))
print(tokenizer_qwen.unk_token)
print(tokenizer_qwen.unk_token_id)
text = "hello world"
print(tokenizer_qwen.encode(text))
print(tokenizer_qwen.encode(' ' + text))

<|endoftext|>
151643
<|endoftext|>
None
None
[23811, 1879]
[220, 23811, 1879]


In [13]:
dataset_qwen = get_dataset_nlp(dataset_name=dataset_name,
                                 mode="eval",
                                 tokenizer=tokenizer_qwen,
                                 batch_size=1,
                                 data_path=f"../datasets/{dataset_name}")
dataset_qwen.load_data()
sentences = np.array(dataset_qwen.dataset_raw["text"])
save_dir_qwen = f"../players/{dataset_name}/players-qwen"

print(len(sentences))

Map: 100%|██████████| 753/753 [00:00<00:00, 965.49 examples/s] 

753


In [14]:
player_words_all_with_idx = load_json(os.path.join(save_dir_qwen, 'player_words.json'))
player_words_all = list(player_words_all_with_idx.values())
sample_indices = [int(s) for s in player_words_all_with_idx.keys()]

assert len(player_words_all) == len(sample_indices) == len(sentences)

player_ids_all = get_player_ids_from_word(tokenizer=tokenizer_qwen,
                                            sentences=sentences[sample_indices],
                                            player_words_all=player_words_all,
                                            save_dir=None)
assert len(player_ids_all) == len(sample_indices)

player_ids_all_with_idx = {f"{idx}": player_ids for idx, player_ids in zip(sample_indices, player_ids_all)}
save_json(player_ids_all_with_idx, save_dir_qwen, 'player_ids_from_word.json')